# Standard Name Table

A standard name table defines standard names. Those names are intended to be used in a file. To describe them propertly, a JSON-LD file describing the standard name table is recommended.

The cf-convention is based on the principle. Let's describe their table:

In [1]:
import ssnolib
from ssnolib.namespace import SSNO

In [2]:
from ssnolib.prov import Person

In [3]:
snt = ssnolib.StandardNameTable(title='SNT from scratch', creator=Person(orcidID="https://orcid.org/0000-0001-8729-0482"))

In [4]:
component = ssnolib.Qualification(name="component", hasValidValues=["x", "y", "z"], description="The component of a vector")

Transformation "component_derivative_of_X"

In [5]:
print(component.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "schema": "http://schema.org/",
        "dcterms": "http://purl.org/dc/terms/"
    },
    "@type": "ssno:Qualification",
    "schema:name": "component",
    "dcterms:description": "The component of a vector",
    "ssno:hasValidValues": [
        "x",
        "y",
        "z"
    ],
    "@id": "_:N68c68936eed14eedb07d16fe0afc9781"
}


In [6]:
transformation = ssnolib.Transformation(
    name="C_derivative_of_X",
    description="derivative of X with respect to distance in the component direction, which may be x, y or z.",
    altersUnit="[X]/[C]",
    hasCharacter=[
        ssnolib.Character(character="X", associatedWith=SSNO.AnyStandardName),
        ssnolib.Character(character="C", associatedWith=component),
    ]
)

In [7]:
snt.definesStandardNameModification = transformation

In [8]:
print(transformation.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "schema": "http://schema.org/",
        "dcterms": "http://purl.org/dc/terms/"
    },
    "@type": "ssno:Transformation",
    "schema:name": "C_derivative_of_X",
    "dcterms:description": "derivative of X with respect to distance in the component direction, which may be x, y or z.",
    "ssno:unitModificationRule": "[X]/[C]",
    "ssno:hasCharacter": [
        {
            "@type": "ssno:Character",
            "ssno:character": "X",
            "ssno:associatedWith": "https://matthiasprobst.github.io/ssno#AnyStandardName",
            "@id": "_:N99f585b80a654572b8c3026f3fe0146b"
        },
        {
            "@type": "ssno:Character",
            "ssno:character": "C",
            "ssno:associatedWith": {
                "@type": "ssno:Qualification",
                "schema:name": "componen

Dump the objects as JSON-LD:

In [9]:
print(snt.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "dcterms": "http://purl.org/dc/terms/",
        "prov": "http://www.w3.org/ns/prov#",
        "ssno": "https://matthiasprobst.github.io/ssno#",
        "foaf": "http://xmlns.com/foaf/0.1/",
        "m4i": "http://w3id.org/nfdi4ing/metadata4ing#",
        "schema": "http://schema.org/"
    },
    "@type": "ssno:StandardNameTable",
    "dcterms:title": "SNT from scratch",
    "dcterms:creator": {
        "@type": "prov:Person",
        "m4i:orcidId": "https://orcid.org/0000-0001-8729-0482",
        "@id": "_:N0035385827d64ff9869d1efcc63683ff"
    },
    "definesStandardNameModification": {
        "@type": "ssno:Transformation",
        "schema:name": "C_derivative_of_X",
        "dcterms:description": "derivative of X with respect to distance in the component direction, which may be x, y or z.",
        "ssno: